# Demo of SpikeShip with neural data

A short example using [AllenSDK](https://alleninstitute.github.io/AllenSDK/install.html) and SpikeShip dissimilarity measure to find temporal spiking patterns in NWB datasets.


We use NWB dataset as an example of using the SpikeShip algorithm. The first step consists in build another dataset with epochs (i.e, trial, time window) and spike times. This new structure follows the work called "SPOTDis" (Grossberger L., et al.) .

It consists of two data-structures:

1) The spike times vector (e.g. `spike_times = [0.2, 0.6, 0.65, 0.9, 1.1, 0.3, 0.9, 0.2]`) of type float with one entry for each spike.

2) The index matrix with the dimensions (#epochs, #neurons, 2) of type int where the last dimension contains a start and a stop index for the spike times vector, indicating which spikes belong to the selected epoch and neuron.

ii_spike_times = [

    # Epoch 1    
    [
        # Neuron 1 -> spike(s) at time(s) 0.2, 0.6
        [0, 2],
        # Neuron 2 -> spike(s) at time(s) 0.65
        [2, 3],
        # Neuron 3 -> spike(s) at time(s) 0.9, 1.1
        [3, 5],
    ],
    
    # Epoch 2
    [
        # Neuron 1 -> spike(s) at time(s) 0.3
        [5, 6],
        # Neuron 2 -> spike(s) at time(s) 0.9
        [6, 7],
        # Neuron 3 -> spike(s) at time(s) 0.2
        [8, 9],
    ],
]

The spike times for a given epoch `e` and neuron `n` can be retrieved from the spike times vector using the index matrix as follows:
```epoch_neuron_spike_times = spike_times[ii_spike_times[e, n, 0]:ii_spike_times[e, n, 1]]
```

Check [SPOTDis Demo](https://github.com/LGro/spot/blob/master/notebooks/SPOTDisClust_Demo.ipynb) for more information.


## Importing libraries

In [ ]:
# tools
import os

import numpy as np
import matplotlib.pyplot as plt

# warnings from jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# distance computation
from spikeship import spikeship

# visualization
import hdbscan
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

---

## Reading data

In [ ]:
spike_times = np.load('../example_data/allen_nwb/spike_times.npy')
ii_spike_times = np.load('../example_data/allen_nwb/ii_spike_times.npy')
stim_labels = np.load('../example_data/allen_nwb/stim_label.npy')

---
## SpikeShip
### Computation

In [ ]:
%%time
# computation of neuron specific-flows
dissimilarity = spikeship.distances(spike_times, ii_spike_times)

### Visualization

#### t-SNE Computation

In [ ]:
tsne_model = TSNE(metric='precomputed', n_components=2)
embedding_dis  = tsne_model.fit_transform(dissimilarity)

#### Dissimilarity matrx of neuron-specific shifts and 2D t-SNE embedding

In [ ]:
stim_values = np.unique(stim_labels)
stim_values

In [ ]:
plt.clf()
fs = 13
alpha = 0.5
marker = 'o'
tPlot, axes = plt.subplots(
    figsize = (10,4),
    nrows=1, ncols=2, sharex=False, sharey=False, 
    constrained_layout=True
    )

# Dissimilarity matrix (vis)
axes[0].set_title('Dissimilarity matrix', fontsize=fs)
np.fill_diagonal(dissimilarity, np.nan) # for visualization purposes
im0 = axes[0].imshow(dissimilarity, cmap='PuBu')
axes[0].set_xlabel('Epoch', fontsize=fs)
axes[0].set_ylabel('Epoch', fontsize=fs);
plt.colorbar(im0, ax=axes[0], label='Neuron-specific flows')

# 2D tsne embedding (vis)
l_colors = ['royalblue', 'orange', 'green', 'purple']
for i in range(len(stim_values)):
    stim_val = stim_values[i]
    mask = (stim_labels == stim_val)
    axes[1].set_title('2D t-SNE Embedding', fontsize=fs)
    axes[1].scatter(embedding_dis[mask, 0], embedding_dis[mask, 1], s=50, label=stim_val)#, c=np.repeat(l_colors[i], np.sum(mask)), alpha=alpha, marker=marker, facecolors='none')
    axes[1].set_xlabel("1st Component", fontsize=fs); axes[1].set_ylabel("2nd Component", fontsize=fs)

axes[1].legend(bbox_to_anchor=(1,1), frameon=False, title="Drifting gratings\n(degrees)")

---

## Comparison with SPOTDis
In order to test the SPOTDis module, follow the instructions in the github repository of its authors: https://github.com/LGro/spot/blob/master/notebooks/SPOTDisClust_Demo.ipynb.

In [ ]:
from spot import spot

In [ ]:
%%time
diss_spot = spot.distances(spike_times, ii_spike_times, metric='SPOTD_xcorr')

#### tSNE Computation

In [ ]:
tsne_model = TSNE(metric='precomputed', n_components=2)
embedding_dis  = tsne_model.fit_transform(diss_spot)

In [ ]:
plt.clf()
fs = 13
alpha = 0.5
marker = 'o'
tPlot, axes = plt.subplots(
    figsize = (10,4),
    nrows=1, ncols=2, sharex=False, sharey=False, 
    constrained_layout=True
    )

# Dissimilarity matrix (vis)
axes[0].set_title('Dissimilarity matrix', fontsize=fs)
np.fill_diagonal(diss_spot, np.nan) # for visualization purposes
im0 = axes[0].imshow(diss_spot, cmap='PuBu')
axes[0].set_xlabel('Epoch', fontsize=fs)
axes[0].set_ylabel('Epoch', fontsize=fs);
plt.colorbar(im0, ax=axes[0], label='SPOT distance')

# 2D tsne embedding (vis)
l_colors = ['royalblue', 'orange', 'green', 'purple']
for i in range(len(stim_values)):
    stim_val = stim_values[i]
    mask = (stim_labels == stim_val)
    axes[1].set_title('2D t-SNE Embedding', fontsize=fs)
    axes[1].scatter(embedding_dis[mask, 0], embedding_dis[mask, 1], s=50, label=stim_val)#, c=np.repeat(l_colors[i], np.sum(mask)), alpha=alpha, marker=marker, facecolors='none')
    axes[1].set_xlabel("1st Component", fontsize=fs); axes[1].set_ylabel("2nd Component", fontsize=fs)

axes[1].legend(bbox_to_anchor=(1,1), frameon=False, title="Drifting gratings\n(degrees)")

In [ ]:
plt.clf()
alpha = 0.5
marker = 'o'
tPlot, axes = plt.subplots(
    figsize = (10,10),
    nrows=2, ncols=2, sharex=False, sharey=False, 
    constrained_layout=True
    )
# dissimilarity matrix (vis)
im0 = axes[0][0].imshow(dissimilarity, cmap='PuBu')
axes[0][0].set_xlabel('Epoch')
axes[0][0].set_ylabel('Epoch');
plt.colorbar(im0, ax=axes[0][0])

# 2D tsne embedding (vis)
axes[1][0].set_title('2D t-SNE Embedding')
axes[1][0].scatter(embedding_dis[:, 0], embedding_dis[:, 1], cmap='Set1_r', s=50,c=cluster_labels_dis, alpha=alpha, marker=marker, facecolors='none')


In order to compute the dissimilarity between two pairs of epochs, SPOTDis has a computational complexity of $\mathcal{O}(N^2n^2)$, where $N$ is the number of active Neurons, and $n$ the average spikes per epoch. In contrast, SpikeShip has a computational complexity of $\mathcal{O}(Nn)$ (Quadratic speed-up).

---

## Preprocessing data from Allen Brain Institute (NWB format)


Download a session as example of NWB data from brain-map API.

In [ ]:
# !wget http://api.brain-map.org//api/v2/well_known_file_download/1026124216

Importing libraries:

In [ ]:
from spikeship import data_formatter
from allensdk.brain_observatory.ecephys.ecephys_session import EcephysSession

Reading a NWB session:

In [ ]:
# Load the session downloaded previouly
session = EcephysSession.from_nwb_path('1026124216')

# Get the spike_times (dict) from the nwb session
nwb_ds = np.array(session.spike_times)

We need to create two vectors: a vector of neuron ids `v_neuron_ids`, and a vector for the spike times `v_spike_times`. Neurons ids have to be renamed as integer values from $0$ to $N-1$, with $N$ the number of neurons.

In [ ]:
# Getting spike times and neuron ids as two separated vectors from NWB dataset
print ("Getting spike times and neuron ids as two separated vectors from NWB dataset")
v_neuron_ids  = [] # vect of neurons
v_spike_times = [] # vect of spike times
c = 0
d_neuron_rename = {}
for neuron_id, spks in nwb_ds.item().items():
    # renaming neuron_ids
    for ii in range(len(nwb_ds.item().keys())):
        if not neuron_id in d_neuron_rename:
            d_neuron_rename[neuron_id] = c
            c += 1
    # compute each spike time for neuron
    for val in spks:
        v_neuron_ids.append( int(neuron_id) )
        v_spike_times.append( float(val) )

v_neuron_ids  = np.array([ d_neuron_rename[neuron] for neuron in v_neuron_ids ] )
v_spike_times = np.array(v_spike_times)

print ('v_neuron_ids ', v_neuron_ids)
print ('v_spike_times', v_spike_times)

### Data Formatting
Definition of number of `epochs` and time window length (i.e., `time_slide_length`).

In this step, we define the local analysis. In this example, we select for the first 1000 seconds of analysis (See variables `ds_min_t` and `ds_max_t`).

Each epoch (i.e., temporal window or trial) has a length of `time_slide_length = 5` seconds.
We use the method called `data_formatter.allocate_spikes` to convert the data into the SPOTDis and SpikeShip data format. This method returns a new dataset with intermediate epochs that contains an overlap between the previous and posterior epoch. For instance, in this scenario the first epoch considers the time from $0.0$ to $3.0$s and the third epoch goes from $3.0$ to $6.0$s. Thus, the second epoch is defined from $1.5$ to $4.5$s.

In [ ]:
# Define local analysis.
ds_min_t = 3               # start point for time window
ds_max_t = 1000 + ds_min_t # end point for time window
time_slide_length = 5      # time window length (5 secs)

# definition of parameters for 'allocate_spikes' method
# number of chunks (epochs)
n_max_chunks = int(np.ceil( (2*(ds_max_t - ds_min_t)) / time_slide_length ) - 1) # total number of chunks or epochs
chunk_increment = time_slide_length / 2. # overlapping
total_spikes  = len(v_spike_times)
total_neurons = len([item for item in nwb_ds.item()])

print ('total_neurons', total_neurons)
print ('total_spikes', total_spikes)
print ('v_spike_times min:', np.min(v_spike_times))
print ('v_spike_times max:', np.max(v_spike_times))
print ('ds_min_t selected:', ds_min_t)
print ('ds_max_t selected:', ds_max_t)
print ('# epochs', n_max_chunks)

In [ ]:
ii_spike_times, spike_times = data_formatter.allocate_spikes(
    v_spike_times, total_spikes,v_neuron_ids, n_max_chunks, chunk_increment,ds_min_t,total_neurons, time_slide_length
)
print('[INFO] total spike_times',len(spike_times), 'total epochs', ii_spike_times.shape[0])

In [ ]:
f, ax = plt.subplots(figsize=(10,5))

ax.set_title('Example pattern epoch raster plot')

i_epoch = 0 # selected epoch for plotting
n_neurons = 100

for i_neuron in range(n_neurons):
    tmp_epoch_neuron_spike_times = spike_times[
        ii_spike_times[i_epoch, i_neuron, 0]:ii_spike_times[i_epoch, i_neuron, 1]]
    ax.scatter(tmp_epoch_neuron_spike_times, np.ones(len(tmp_epoch_neuron_spike_times))+i_neuron,
               marker='|', c='black')

ax.set_yticks(np.arange(0, n_neurons+1, 5))
ax.set_ylabel('# Neuron')
ax.set_xlabel('Time');

---
Check out our paper for more details:

Sotomayor-Gómez, B., Battaglia, F. P., & Vinck, M. (2021). [SpikeShip: A method for fast, unsupervised discovery of high-dimensional neural spiking patterns](https://doi.org/10.1101/2020.06.03.131573). *bioRxiv*.

---